<a href="https://colab.research.google.com/github/changyong93/Natural-language-processing-with-chat-bot/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%85%EB%AC%B8(12_4_NLP%EB%A5%BC_%EC%9C%84%ED%95%9C_%EC%8B%A0%EA%B2%BD%EB%A7%9D(CNN)_1D_CNN%EC%9C%BC%EB%A1%9C_%EC%8A%A4%ED%8C%B8_%EB%A9%94%EC%9D%BC_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1D CNN으로 스팸 메일 분류하기

In [ ]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")

In [ ]:
data = pd.read_csv("spam.csv",encoding = 'latin-1')
data.shape

In [ ]:
data[:5]

In [ ]:
data = data.iloc[:,:2].copy()
data["v1"] = data.v1.replace(["ham","spam"],[0,1])

In [ ]:
data[:5]

In [ ]:
data.v1.nunique(), data.v2.nunique(), data.shape[0]

In [ ]:
data = data.drop_duplicates(subset = ["v2"]).copy()

In [ ]:
len(data)

In [ ]:
data.v1.value_counts().plot.bar()

In [ ]:
data.v1.value_counts()

In [ ]:
X_data = data.v2
y_data = data.v1

len(X_data), len(y_data)

In [ ]:
vocab_size = 1000
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_data)

In [ ]:
sequence = tokenizer.texts_to_sequences(X_data)
sequence[:5]

In [ ]:
train_idx = int(len(X_data) * 0.8)
print(f"train / test 개수: {train_idx} / {len(X_data) - train_idx}")

In [ ]:
X_data = sequence
print(f"메일의 최대 / 평균 길이 : [{max([len(s) for s in X_data])} / {np.mean([len(s) for s in X_data])}]")
plt.hist([len(s) for s in X_data],bins = 50)

In [ ]:
max_len = max([len(s) for s in X_data])
X_data = pad_sequences(sequences=X_data, maxlen = max_len)
X_data.shape

In [ ]:
X_train = X_data[:train_idx]
X_test = X_data[train_idx:]
y_train = np.array(y_data[:train_idx])
y_test = np.array(y_data[train_idx:])

X_train.shape, X_test.shape, y_train.shape, y_test.shape

## 1D Convolution

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPool1D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
model = Sequential()

model.add(Embedding(input_dim = vocab_size, output_dim = 32))
model.add(Dropout(rate = 0.2))
model.add(Conv1D(filters = 32, kernel_size = 3, strides = 1,padding = 'valid',activation = 'relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(units = 64, activation = "relu"))
model.add(Dropout(rate = 0.2))
model.add(Dense(units = 1, activation = "sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

es = EarlyStopping(monitor = 'val_loss', verbose = 1, mode = 'min')
mc = ModelCheckpoint(filepath = "base_model.h5",monitor = 'val_acc',mode = 'max',save_best_only=True)

history = model.fit(x = X_train, y = y_train, epochs = 10, batch_size = 64, validation_split=0.2, callbacks=[es,mc])

In [ ]:
loaded_model = load_model("base_model.h5")
print("\n 테스트 정확도: ", loaded_model.evaluate(x = X_test, y = y_test))